# PUBG Finish Placement Prediction (Kernels Only)

- 초보의 연습입니다. 매우 짧으며 설명이 없을 수도 있습니다.
- 분석이 적을 수도 있으며 빈약할 수 있습니다.
- 재미있게 봐주세요! 모두 공부 화이팅!
- 이 글은 SeaOtter, DIMITRIOS EFFROSYNIDIS님의 노트북을 참고했습니다.

## 1. 대회설명

- 플레이어 등급의 최종 배치를 예측하시오!
- PUBG에서 승리하기 위한 최고의 전략은 존버메타가 정답일까요? 아니면 여포처럼 휘둘러 다녀야 할까요?? 데이터로 말해봅시다!!

## 2. 게임 설명

- 다들 배그 한번씩 해보셨나요? 배그는 돌아다니며 무기, 총알, 방어구, 구급약 등을 수집하며 필드 속에서 서로를 죽이며 최후까지 살아남는 게임입니다.

- 맨 처음 비행기에서 원하는 위치에서 떨어져 파밍(아이템을 줍는 행위)을 하며 싸우기 위한 준비를 합니다.

- 사플이라고하죠.. 이 게임으로 인해 전국 피시방에 자리마다 헤드셋이 보급되었습니다.. 그 만큼 소리에 민감한 게임입니다.

- 건물에 숨어서 적을 죽이는 존버라는 말과 차 소리, 총소리를 내며 사람들을 학살 하는 여포라는 말이 유행하게 되었습니다.

In [ ]:
import pandas as pd #Analysis 
import matplotlib.pyplot as plt #Visulization
import seaborn as sns #Visulization
import numpy as np #Analysis 
from scipy.stats import norm #Analysis 
from sklearn.preprocessing import StandardScaler #Analysis 
from scipy import stats #Analysis 
import warnings 
warnings.filterwarnings('ignore')
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.figure_factory as ff
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
import gc

In [ ]:
#If you run all dataset, you change debug False
debug = False
if debug == True:
    df_train = pd.read_csv('../input/pubg-finish-placement-prediction/train_V2.csv', nrows=10000)
    df_test  = pd.read_csv('../input/pubg-finish-placement-prediction/test_V2.csv')
else:
    df_train = pd.read_csv('../input/pubg-finish-placement-prediction/train_V2.csv')
    df_test  = pd.read_csv('../input/pubg-finish-placement-prediction/test_V2.csv')

In [ ]:
print("Train : ", df_train.shape)
print("Test : ", df_test.shape)

In [ ]:
df_train.info()

In [ ]:
df_train.head()

아래의 코드는 메모리를 최적화 하는 코드입니다..
저도 뭔지는 잘 모릅니다!

In [ ]:
# Memory saving function credit to https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    #start_mem = df.memory_usage().sum() / 1024**2
    #print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    #end_mem = df.memory_usage().sum() / 1024**2
    #print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    #print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df
df_train = reduce_mem_usage(df_train)
df_test = reduce_mem_usage(df_test)

## 3. 변수 설명

#### groupId, matchId의 차이점은??

- groupId : 동일한 그룹인지 확인하는 Id입니다. 같은 그룹일지라도 다른 match에는 다른 groubid가 사용됩니다.

- matchId : match를 식별하기 위한 Id입니다. traning set과 testing set 모두 같은 match는 없습니다.

In [ ]:
df_train[df_train['groupId']=='4d4b580de459be']

In [ ]:
len(df_train[df_train['matchId']=='a10357fd1a4a91'])

- 위의 두가지 예를 보면, groupId가 4d4b580de459be로 동일한 4명은 matchId가 a10357fd1a4a91로 동일하기에 같은 match에서 4인큐를 돌렸다고 생각하면 됩니다.

- matchId가 a10357fd1a4a91인 사람 수는 96명으로 한 게임 내에 96명이 참가했다는 뜻 입니다. 배틀그라운드 게임은 한판에 대략 100명 정도가 참여합니다.

In [ ]:
temp = df_train[df_train['matchId']=='a10357fd1a4a91']['groupId'].value_counts().sort_values(ascending=False)
#print("Total number of states : ",len(temp))
trace = go.Bar(
    x = temp.index,
    y = (temp)
)
data = [trace]
layout = go.Layout(
    title = "GroupId of Match Id: a10357fd1a4a91",
    xaxis=dict(
        title='groupId',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Count of groupId of type of MatchId a10357fd1a4a91',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
)
)
fig = go.Figure(data=data, layout=layout)
fig

- matchId가 a10357fd1a4a91라는 한판 내에서 groupId가 같은 명수를 세봤는데 4이상인 수가 나왔습니다.

- 배틀그라운드는 4인큐가 제일 많이하는 것인데 그 이상인 수가 나왔습니다.

- michaelapers님이 말하길 : 게임에서 연결이 끊기기 때문에 발생하는 현상입니다. 연결 해제가 발생하면 여러 그룹의 플레이어가 API의 데이터베이스에 동일한 최종 배치를 가진 것으로 저장되기 때문입니다.

- 그렇기에 우리는 groupId를 확실히 함께한 팀이 아니라 동일한 최종 순위를 가진 선수라고 생각해야 합니다.

#### Solos, Duo, Squads
- 혼자 플레이하면 솔로, 둘이 함께하면 듀오, 네명이서 하면 스쿼드

In [ ]:
solos = df_train[df_train['numGroups']>50]
duos = df_train[(df_train['numGroups']>25) & (df_train['numGroups']<=50)]
squads = df_train[df_train['numGroups']<=25]
f,ax1 = plt.subplots(figsize =(20,10))
sns.pointplot(x='kills',y='winPlacePerc',data=solos,color='black',alpha=0.8)
sns.pointplot(x='kills',y='winPlacePerc',data=duos,color='#CC0000',alpha=0.8)
sns.pointplot(x='kills',y='winPlacePerc',data=squads,color='#3399FF',alpha=0.8)
plt.text(37,0.6,'Solos',color='black',fontsize = 17,style = 'italic')
plt.text(37,0.55,'Duos',color='#CC0000',fontsize = 17,style = 'italic')
plt.text(37,0.5,'Squads',color='#3399FF',fontsize = 17,style = 'italic')
plt.xlabel('Number of kills',fontsize = 15,color='blue')
plt.ylabel('Win Percentage',fontsize = 15,color='blue')
plt.title('Solo vs Duo vs Squad Kills',fontsize = 20,color='blue')
plt.grid() #줄금표시
plt.show()

- 확실히 솔로, 듀오, 스쿼드 순으로 승률이 좋습니다.

- 솔로와 듀오는 같은 맥락의 킬을 가지고 있는 것 같습니다.

#### 데이터 세부 정보 설명

#### assists 
- 내가 상대방을 죽이지 않았지만 상대방을 죽이는 것을 도왔을 때 스코어가 오릅니다! 직접적인 가해가 있어야지 스코어가 오릅니다! kill스코어와는 다릅니다.

In [ ]:
temp = df_train['assists'].value_counts().sort_values(ascending=False)

#print("Total number of states : ",len(temp))
trace = go.Bar(
    x = temp.index,
    y = (temp)
)
data = [trace]
layout = go.Layout(
    title = "",
    xaxis=dict(
        title='assists',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Count of assists',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
)
)
fig = go.Figure(data=data, layout=layout)
fig

In [ ]:
temp = df_train['assists'].value_counts().sort_values(ascending=False)

#print("Total number of states : ",len(temp))
trace = go.Bar(
    x = temp.index,
    y = (temp)
)
data = [trace]
layout = go.Layout(
    title = "",
    xaxis=dict(
        title='assists',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Count of assists',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
)
)
fig = go.Figure(data=data, layout=layout)

fig.update_xaxes(range=[0, 10])
fig.update_yaxes(range=[0, 150000])

fig

- 대부분 어시스트가 없습니다!
- 있다면 1~3 정도가 있군요!
- 0, 1 어시스트는 15만 이상으로 생각하시면 됩니다.

#### Kills

- 상대방을 죽인 횟수입니다.

In [ ]:
temp = df_train['kills'].value_counts().sort_values(ascending=False)

#print("Total number of states : ",len(temp))
trace = go.Bar(
    x = temp.index,
    y = (temp)
)
data = [trace]
layout = go.Layout(
    title = "",
    xaxis=dict(
        title='kills',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Count of kills',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
)
)
fig = go.Figure(data=data, layout=layout)

fig.update_xaxes(range=[0, 20])
fig.update_yaxes(range=[0, 400000])

fig

In [ ]:
df_train['kills'].mean()

In [ ]:
df_train['kills'].quantile(0.99)

In [ ]:
df_train['kills'].max()

- 평균적으로 0.92킬을 합니다.

- 99퍼센트의 사람들이 7킬 이하를 합니다.

- 가장 많이 기록된 킬은 72킬입니다.

In [ ]:
sns.jointplot(x="winPlacePerc", y="kills", data=df_train, height=10, ratio=3, color="r")
plt.show()

In [ ]:
kills = df_train.copy() #https://docs.python.org/ko/3/library/copy.html

kills['killsCategories'] = pd.cut(kills['kills'], [-1, 0, 2, 5, 10, 80], labels=['0_kills','1-2_kills', '3-5_kills', '6-10_kills', '10+_kills'])

plt.figure(figsize=(15,8))
sns.boxplot(x="killsCategories", y="winPlacePerc", data=kills)
plt.show()

 확실히 킬을 많이 할수록 승률이 올라가긴 합니다!!

#### killStreaks

- 짧은 시간안에 상대방을 죽인 횟수.

In [ ]:
temp = df_train['killStreaks'].value_counts().sort_values(ascending=False)

#print("Total number of states : ",len(temp))
trace = go.Bar(
    x = temp.index,
    y = (temp)
)
data = [trace]
layout = go.Layout(
    title = "",
    xaxis=dict(
        title='killStreaks',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Count of killStreaks',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
)
)
fig = go.Figure(data=data, layout=layout)
fig

In [ ]:
temp = df_train['killStreaks'].value_counts().sort_values(ascending=False)

#print("Total number of states : ",len(temp))
trace = go.Bar(
    x = temp.index,
    y = (temp)
)
data = [trace]
layout = go.Layout(
    title = "",
    xaxis=dict(
        title='killStreaks',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Count of killStreaks',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
)
)
fig = go.Figure(data=data, layout=layout)

fig.update_xaxes(range=[0, 10])
fig.update_yaxes(range=[0, 400000])

fig

- 킬을 한 사람 중에서 대부분이 1킬만 하고 끝납니다.
- 연달아 킬을 한 경우는 더블킬이 제일 많고 간간히 트리플킬까지 있습니다.
- 배그의 특성상 연달아 킬을 하는 경우는 같은 지역에 몰려있는 경우인데 몰려있다면 한 그룹을 만나게 되는 것이니 최대 4명정도가 몰려있다고 생각하니 이 수치가 어느정도 들어 맞는 것 같습니다.

In [ ]:
sns.jointplot(x="winPlacePerc", y="killStreaks", data=df_train, height=10, ratio=3, color="r")
plt.show()

#### RoadKills
- 차량을 타고 상대방을 치여 죽인 스코어 입니다.
- 흔히 고라니킬 이라고 합니다.

In [ ]:
temp = df_train['roadKills'].value_counts().sort_values(ascending=False)

#print("Total number of states : ",len(temp))
trace = go.Bar(
    x = temp.index,
    y = (temp)
)
data = [trace]
layout = go.Layout(
    title = "",
    xaxis=dict(
        title='roadKills',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Count of roadKills',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
)
)
fig = go.Figure(data=data, layout=layout)
fig

In [ ]:
temp = df_train['roadKills'].value_counts().sort_values(ascending=False)

#print("Total number of states : ",len(temp))
trace = go.Bar(
    x = temp.index,
    y = (temp)
)
data = [trace]
layout = go.Layout(
    title = "",
    xaxis=dict(
        title='roadKills',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Count of roadKills',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
)
)
fig = go.Figure(data=data, layout=layout)

fig.update_xaxes(range=[0, 5])
fig.update_yaxes(range=[0, 15000])

fig

In [ ]:
sns.jointplot(x="winPlacePerc", y="roadKills", data=df_train, height=10, ratio=3, color="r")
plt.ylim(0,4)
plt.show()

- 역시 0킬이 압도적으로 많습니다. 배그의 특성상 차량을 타면 소리가 나서 본인의 위치를 들키기에 로드킬을 하는 경우가 적은 것으로 알 수 있습니다.

- 1킬 정도는 지나다니면서 할 수는 있지만 2킬부터 급격히 줄어든 모습을 볼 수 있습니다.

#### Teamkills
- 팀을 죽인 횟수입니다.

In [ ]:
temp = df_train['teamKills'].value_counts().sort_values(ascending=False)

#print("Total number of states : ",len(temp))
trace = go.Bar(
    x = temp.index,
    y = (temp),
)
data = [trace]
layout = go.Layout(
    title = "",
    xaxis=dict(
        title='teamKills',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Count of teamKills',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
)
)
fig = go.Figure(data=data, layout=layout)
fig

In [ ]:
temp = df_train['teamKills'].value_counts().sort_values(ascending=False)

#print("Total number of states : ",len(temp))
trace = go.Bar(
    x = temp.index,
    y = (temp),
)
data = [trace]
layout = go.Layout(
    title = "",
    xaxis=dict(
        title='teamKills',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Count of teamKills',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
)
)
fig = go.Figure(data=data, layout=layout)

fig.update_xaxes(range=[0, 4])
fig.update_yaxes(range=[0, 10000])

fig

- 팀킬은 이해할 수 없는 행동입니다. 그러나 팀을 1명 죽이는 경우가 가장 많습니다.

- 이러한 경우는 왜 일어날까요?

1. 재미를 위해 죽인다.
1. 팀이 너무 못해서 죽인다.
1. 3명이서 4인 스쿼드를 돌릴경우 1명은 랜덤으로 만나기에 그냥 죽인다.
1. 혼자서 스쿼드를 돌리고 3명을 죽인 뒤, 혼자 상대 스쿼드를 격파하고 싶어서.

#### LongestKill
- 사망 시 사망한 플레이어와 본인의 거리입니다.

In [ ]:
f, ax = plt.subplots(figsize=(18, 8))
sns.distplot(df_train['longestKill'])

In [ ]:
f, ax = plt.subplots(figsize=(18, 8))
plt.xlim(0, 200)
sns.distplot(df_train['longestKill'])

y축은 밀도, x축은 거리입니다.

- 대부분 0~25m이내에서 킬이 일어났습니다.
- 간혹 비정상적으로 거리가 멀 때 킬이 기록되는데 이 때 상대를 기절 시키고 차량으로 멀리 간 후, 상대방이 소생하지 못하여 죽었을 경우도 기록이 되는 것으로 보입니다.

#### WeaponsAcquired
- 무기를 줍는 횟수입니다.

In [ ]:
temp = df_train['weaponsAcquired'].value_counts().sort_values(ascending=False)

#print("Total number of states : ",len(temp))
trace = go.Bar(
    x = temp.index,
    y = (temp),
)
data = [trace]
layout = go.Layout(
    title = "",
    xaxis=dict(
        title='weaponsAcquired',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Count of weaponsAcquired',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
)
)
fig = go.Figure(data=data, layout=layout)

fig.update_xaxes(range=[0, 15])

fig

- 무기를 2~3번 정도 줍는 것이 가장 많은 것으로 보입니다.

#### HeadshotKills - not knocked
- 머리를 관통해서 상대방이 즉시 사망한 경우를 말합니다. 여기서 중요한 것은 '즉시' 사망했다는 것입니다.

In [ ]:
temp = df_train['headshotKills'].value_counts().sort_values(ascending=False)

#print("Total number of states : ",len(temp))
trace = go.Bar(
    x = temp.index,
    y = (temp),
)
data = [trace]
layout = go.Layout(
    title = "",
    xaxis=dict(
        title='headshotKills',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Count of headshotKills',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
)
)
fig = go.Figure(data=data, layout=layout)

fig.update_xaxes(range=[0, 5])

fig

- 대부분 헤드샷을 한번도 못하고 게임이 끝납니다.
- 헤드샷을 했을 경우 1번정도가 절반이상으로 많습니다.

#### DBNOs
- 상대방이 총을 맞고 체력을 모두 잃었을 경우 바닥에 기어다니며 팀원들의 소생을 기다리는 경우를 말합니다.

In [ ]:
temp = df_train['DBNOs'].value_counts().sort_values(ascending=False)

#print("Total number of states : ",len(temp))
trace = go.Bar(
    x = temp.index,
    y = (temp),
)
data = [trace]
layout = go.Layout(
    title = "",
    xaxis=dict(
        title='DBNOs',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Count of DBNOs',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
)
)
fig = go.Figure(data=data, layout=layout)

fig.update_xaxes(range=[0, 5])

fig

- 의외로 0이 많아서 놀랐습니다. 0은 헤드샷을 당했거나 솔로로 플레이 했을 경우 혹은 팀원 중 마지막에 살아있는 경우를 말하는데...

#### Boost vs heal
-부스트는 즉시 치유를 하고 힐은 천천히 체력을 회복시킵니다.

In [ ]:
df_train['heals'].mean()

In [ ]:
df_train['heals'].quantile(0.99)

In [ ]:
df_train['heals'].max()

- 평균적으로 1.3번 힐을 사용한다.

- 99%의 사람들은 12번 힐을 사용한다.

- 최대로 많이 사용한 힐의 개수는 80개이다.

In [ ]:
df_train['boosts'].mean()

In [ ]:
df_train['boosts'].quantile(0.99)

In [ ]:
df_train['boosts'].max()

- 평균적으로 1.1번의 부스트를 사용한다.

- 99%의 사람이 7번의 부스트를 사용한다.

- 최대로 부스트를 많이 쓴 개수는 33개이다.

In [ ]:
temp = df_train['boosts'].value_counts().sort_values(ascending=False)

#print("Total number of states : ",len(temp))
trace = go.Bar(
    x = temp.index,
    y = (temp),
)
data = [trace]
layout = go.Layout(
    title = "",
    xaxis=dict(
        title='boosts',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Count of boosts',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
)
)
fig = go.Figure(data=data, layout=layout)

fig.update_xaxes(range=[0, 10])

fig

In [ ]:
sns.jointplot(x="winPlacePerc", y="boosts", data=df_train, height=10, ratio=3, color="blue")
plt.show()

- 부스트를 먹지 않고 죽는 경우가 많습니다. 초반 킬에 이런 경우가 매우 많은 것 같습니다.

In [ ]:
temp = df_train['heals'].value_counts().sort_values(ascending=False)

#print("Total number of states : ",len(temp))
trace = go.Bar(
    x = temp.index,
    y = (temp),
)
data = [trace]
layout = go.Layout(
    title = "",
    xaxis=dict(
        title='heals',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Count of heals',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
)
)
fig = go.Figure(data=data, layout=layout)

fig.update_xaxes(range=[0, 15])

fig

In [ ]:
sns.jointplot(x="winPlacePerc", y="heals", data=df_train, height=10, ratio=3, color="lime")
plt.show()

- 회복을 즉시 시켜주는 부스트와 천천히 회복시키는 힐은 비슷한 양상을 보이고 있습니다.

### DamageDealt
- 가한 총 피해량 입니다.

In [ ]:
#histogram
f, ax = plt.subplots(figsize=(8, 6))
plt.xlim(0,1000)
sns.distplot(df_train['damageDealt'])

In [ ]:
sns.jointplot(x="winPlacePerc", y="damageDealt", data=df_train, height=10, ratio=3, color="r")
plt.show()

#### revives
- 팀원을 소생시킨 횟수입니다. 헤드샷을 맞지 않고 체력이 모두 깎일 경우 기절상태가 될 때 살릴 수 있습니다.

In [ ]:
f, ax = plt.subplots(figsize=(8, 6))
df_train['revives'].value_counts().sort_values(ascending=False).plot.bar()
plt.xlim(0,4)
plt.show()

- 팀을 안살리는 경우가 매우 많습니다.

#### WalkDistance
- 도보로 이동한 총 거리입니다.

In [ ]:
df_train['walkDistance'].mean()

In [ ]:
np.isnan(df_train['walkDistance']).sum()

In [ ]:
df_train['walkDistance'].dtypes

In [ ]:
df_train['walkDistance'] = df_train['walkDistance'].astype(float)
df_train['walkDistance'].dtype

In [ ]:
df_train['walkDistance'].mean()
#위 문제 해결을 도와준 강인영님께 감사를 올립니다.

In [ ]:
df_train['walkDistance'].quantile(0.99)

In [ ]:
df_train['walkDistance'].max()

- 평균적으로 1154.2m를 걷고

- 99%의 사람들은 4396.0m이하를 걷고

- 제일 많이 걷는 사람은 25780m를 걸었습니다.

In [ ]:
#histogram
f, ax = plt.subplots(figsize=(18, 8))
plt.xlim(0, 5000)
sns.distplot(df_train['walkDistance'])

In [ ]:
sns.jointplot(x="winPlacePerc", y="walkDistance",  data=df_train, height=10, ratio=3, color="lime")
plt.show()

분명히 걷는 이동량은 승률과 높은 상관 관계가 있습니다.

#### RideDistance
- 차량을 타고 이동한 총 거리입니다.

In [ ]:
df_train['rideDistance'] = df_train['rideDistance'].astype(float)
df_train['rideDistance'].mean()

In [ ]:
df_train['rideDistance'].quantile(0.99)

In [ ]:
df_train['rideDistance'].max()

- 평균적으로 606m를 운전합니다.

- 99%의 사람들은 6968m이하를 운전합니다.

- 제일 많이 운전한 사람은 40700m를 이동했습니다.

In [ ]:
#histogram
f, ax = plt.subplots(figsize=(18, 8))
plt.ylim(0, 0.001)
plt.xlim(0, 10000)
sns.distplot(df_train['rideDistance'])

#### VehicleDestroys
-파괴된 차량의 수 입니다.

In [ ]:
temp = df_train['vehicleDestroys'].value_counts().sort_values(ascending=False)

#print("Total number of states : ",len(temp))
trace = go.Bar(
    x = temp.index,
    y = (temp),
)
data = [trace]
layout = go.Layout(
    title = "",
    xaxis=dict(
        title='vehicleDestroys',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Count of vehicleDestroys',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
)
)
fig = go.Figure(data=data, layout=layout)

fig.update_xaxes(range=[0, 2])

fig

- 거의 대부분 차를 부시지 않습니다. 총알 낭비이면서 이득이 없는 행동이기 때문입니다. 그러면 차를 왜 뿌실까요?

1. 상대방이 차에 타고 있을 경우
1. 상대방이 차 뒤에 숨어있을 경우
1. 그냥 재미로 <- 단순해 보이지만 매우 중요한 이유 중 하나입니다. 게임을 하면서 이상한 사람을 매우 많이 만나기 때문입니다.

#### The Swimmers
-수영하는 사람입니다.

In [ ]:
df_train['swimDistance'] = df_train['swimDistance'].astype(float)
df_train['swimDistance'].mean()

In [ ]:
df_train['swimDistance'].quantile(0.99)

In [ ]:
df_train['swimDistance'].max()

- 평균 수영거리는 4.1m입니다.

- 99%의 사람들은 123m 이하를 수영합니다.

- 가장 많이 수영한 거리는 3824m입니다.

In [ ]:
data = df_train.copy()
data = data[data['swimDistance'] < df_train['swimDistance'].quantile(0.95)]
plt.figure(figsize=(15,10))
plt.title("Swim Distance Distribution",fontsize=15)
sns.distplot(data['swimDistance'])
plt.xlim(0,10)
plt.show()

대부분 수영을 하지 않습니다..

In [ ]:
swim = df_train.copy()

swim['swimDistance'] = pd.cut(swim['swimDistance'], [-1, 0, 5, 20, 5286], labels=['0m','1-5m', '6-20m', '20m+'])

plt.figure(figsize=(15,8))
sns.boxplot(x="swimDistance", y="winPlacePerc", data=swim)
plt.show()

- 수영을 많이하면 승리하는 확률이 높아지긴 한다!

In [ ]:
k = 5 #number of variables for heatmap
f,ax = plt.subplots(figsize=(11, 11))
cols = df_train.corr().nlargest(k, 'winPlacePerc')['winPlacePerc'].index
cm = np.corrcoef(df_train[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()